In [33]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors, cm
import kmapper as km
import graphviz as gz
import itertools
import colorsys
import sklearn

np.random.seed(1234)

%matplotlib inline

In [34]:
df=pd.read_csv("DetailedPlayerStats.csv", encoding='ANSI')

C:\Users\noela\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4,10,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
df.head()

,Id,WebName,FirstName,SecondName,DreamTeam,SelectedBy,Position,Team,FixtureWeek1,Opponent1,...,Tackles,AttemptedPasses,PassesCompleted,BigChancesMissed,ErrorsToGoal,ErrorsToGoalAttempt,Tackled,Offside,Fouls,Dribbles
0,1.0,Ospina,David,Ospina,False,0.2,Goalkeeper,Arsenal,Gameweek 34,Newcastle,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,Ospina,David,Ospina,False,0.2,Goalkeeper,Arsenal,Gameweek 34,Newcastle,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,Ospina,David,Ospina,False,0.2,Goalkeeper,Arsenal,Gameweek 34,Newcastle,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,Ospina,David,Ospina,False,0.2,Goalkeeper,Arsenal,Gameweek 34,Newcastle,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,Ospina,David,Ospina,False,0.2,Goalkeeper,Arsenal,Gameweek 34,Newcastle,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df.columns

Index(['Id', 'WebName', 'FirstName', 'SecondName', 'DreamTeam', 'SelectedBy',
       'Position', 'Team', 'FixtureWeek1', 'Opponent1', 'HomeAway1',
       'FixtureWeek2', 'Opponent2', 'HomeAway2', 'FixtureWeek3', 'Opponent3',
       'HomeAway3', 'GameWeek', 'Points', 'Cost', 'NetTransfers',
       'MinutesPlayed', 'Goals', 'Assists', 'CleanSheet', 'Bonus',
       'YellowCards', 'RedCards', 'GoalsConceded', 'Crosses',
       'BigChancesCreated', 'ClearancesBlocksIntercepts', 'Recoveries',
       'KeyPasses', 'Tackles', 'AttemptedPasses', 'PassesCompleted',
       'BigChancesMissed', 'ErrorsToGoal', 'ErrorsToGoalAttempt', 'Tackled',
       'Offside', 'Fouls', 'Dribbles'],
      dtype='object')

In [37]:
%%capture
df.drop(df.columns[[0,2,3,4,20]], axis=1, inplace=True)
df.drop(df.columns[[x for x in range(3,14)]], axis=1, inplace=True)

In [38]:
df.columns

Index(['WebName', 'SelectedBy', 'Position', 'Points', 'Cost', 'MinutesPlayed',
       'Goals', 'Assists', 'CleanSheet', 'Bonus', 'YellowCards', 'RedCards',
       'GoalsConceded', 'Crosses', 'BigChancesCreated',
       'ClearancesBlocksIntercepts', 'Recoveries', 'KeyPasses', 'Tackles',
       'AttemptedPasses', 'PassesCompleted', 'BigChancesMissed',
       'ErrorsToGoal', 'ErrorsToGoalAttempt', 'Tackled', 'Offside', 'Fouls',
       'Dribbles'],
      dtype='object')

In [39]:
g=df.groupby(u'WebName')
df2=g.aggregate(np.mean).reset_index()
df2.head()

,WebName,SelectedBy,Points,Cost,MinutesPlayed,Goals,Assists,CleanSheet,Bonus,YellowCards,...,Tackles,AttemptedPasses,PassesCompleted,BigChancesMissed,ErrorsToGoal,ErrorsToGoalAttempt,Tackled,Offside,Fouls,Dribbles
0,Aarons,0.1,0.15625,43.656250,4.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.125000,1.281250,0.875000,0.000000,0.000000,0.000000,0.250000,0.031250,0.031250,0.125000
1,Abraham,3.5,2.53125,56.343750,49.468750,0.156250,0.093750,0.125000,0.343750,0.000000,...,0.218750,8.906250,6.156250,0.125000,0.000000,0.000000,1.968750,0.250000,0.437500,0.812500
2,Adam,0.1,0.30303,47.545455,12.272727,0.000000,0.000000,0.060606,0.000000,0.060606,...,0.090909,7.424242,4.787879,0.030303,0.030303,0.030303,0.151515,0.000000,0.242424,0.333333
3,Adam Smith,0.5,1.69697,48.303030,62.636364,0.030303,0.090909,0.060606,0.090909,0.181818,...,0.848485,27.272727,22.545455,0.000000,0.000000,0.000000,0.969697,0.030303,0.696970,0.848485
4,Adarabioyo,0.0,0.00000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
df2.describe()

,SelectedBy,Points,Cost,MinutesPlayed,Goals,Assists,CleanSheet,Bonus,YellowCards,RedCards,...,Tackles,AttemptedPasses,PassesCompleted,BigChancesMissed,ErrorsToGoal,ErrorsToGoalAttempt,Tackled,Offside,Fouls,Dribbles
count,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,...,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000
mean,2.369604,1.350845,51.996895,32.213366,0.043877,0.039299,0.107396,0.104787,0.049378,0.001718,...,0.270101,14.713883,11.581061,0.032924,0.005243,0.009432,0.524009,0.069610,0.338840,0.317568
std,5.203923,1.357213,12.659954,29.304038,0.097014,0.070455,0.117102,0.159544,0.062918,0.007421,...,0.319000,15.721768,13.219802,0.073055,0.016333,0.022205,0.741553,0.156641,0.380249,0.487391
min,0.000000,0.000000,39.093750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.100000,0.062500,44.420168,1.369318,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.600379,0.468750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.400000,1.093750,48.406250,30.750000,0.000000,0.000000,0.090909,0.020833,0.030303,0.000000,...,0.156250,11.625000,8.107692,0.000000,0.000000,0.000000,0.187500,0.000000,0.218750,0.093750
75%,2.300000,2.261364,54.476799,55.197443,0.060606,0.060606,0.181818,0.156250,0.090909,0.000000,...,0.454545,24.092330,18.423059,0.031250,0.000000,0.000000,0.784564,0.062500,0.593750,0.437500
max,56.600000,8.272727,127.562500,90.000000,0.878788,0.531250,0.500000,0.857143,0.272727,0.062500,...,1.909091,90.187500,82.781250,0.606061,0.187500,0.156250,4.545455,1.062500,2.125000,4.156250


In [46]:
df2[ df2['Points'] > 5 ]

,WebName,SelectedBy,Points,Cost,MinutesPlayed,Goals,Assists,CleanSheet,Bonus,YellowCards,...,Tackles,AttemptedPasses,PassesCompleted,BigChancesMissed,ErrorsToGoal,ErrorsToGoalAttempt,Tackled,Offside,Fouls,Dribbles
8,Agüero,13.9,5.281250,116.718750,61.250000,0.656250,0.187500,0.406250,0.687500,0.062500,...,0.156250,19.656250,16.250000,0.406250,0.0,0.0,2.250000,0.500000,0.312500,1.718750
26,Aubameyang,13.8,6.428571,105.428571,87.142857,0.857143,0.142857,0.285714,0.857143,0.000000,...,0.428571,18.714286,13.714286,0.571429,0.0,0.0,0.714286,1.000000,0.142857,0.857143
134,De Bruyne,23.3,5.906250,101.718750,83.593750,0.218750,0.531250,0.500000,0.781250,0.031250,...,0.781250,73.687500,61.718750,0.093750,0.0,0.0,2.031250,0.062500,0.843750,1.687500
164,Eriksen,16.2,5.531250,94.312500,83.687500,0.312500,0.281250,0.468750,0.718750,0.000000,...,0.593750,58.562500,48.218750,0.062500,0.0,0.0,2.656250,0.031250,0.375000,0.687500
178,Firmino,43.6,5.030303,88.454545,72.212121,0.424242,0.242424,0.393939,0.818182,0.030303,...,1.090909,30.181818,21.757576,0.151515,0.0,0.0,2.515152,0.303030,1.545455,1.333333
279,Kane,31.0,5.656250,127.562500,79.375000,0.750000,0.062500,0.406250,0.843750,0.156250,...,0.375000,16.906250,12.218750,0.468750,0.0,0.0,3.031250,1.062500,0.968750,1.218750
342,Mahrez,13.9,5.187500,85.500000,75.250000,0.312500,0.343750,0.250000,0.562500,0.031250,...,0.343750,32.875000,26.031250,0.125000,0.0,0.0,3.656250,0.312500,0.625000,2.062500
490,Salah,56.6,8.272727,98.212121,74.818182,0.878788,0.333333,0.363636,0.787879,0.000000,...,0.181818,25.151515,19.515152,0.545455,0.0,0.0,3.636364,0.484848,0.424242,2.030303
520,Son,18.2,5.031250,80.281250,62.437500,0.375000,0.187500,0.375000,0.500000,0.000000,...,0.281250,25.406250,21.687500,0.250000,0.0,0.0,2.375000,0.750000,0.406250,1.781250


In [47]:
mapper = km.KeplerMapper(verbose=0)
lens = mapper.fit_transform(df2.drop('WebName', axis=1), projection=sklearn.manifold.TSNE)
#sklearn.manifold.TSNE
#plt.hist(lens)

In [48]:
graph = mapper.map(lens, df2.drop('WebName', axis=1), nr_cubes=200, overlap_perc=0.7)

KeyboardInterrupt: 

In [ ]:
network=gz.Graph()
network.attr(layout='neato')
network.attr('edge', len='0.7')
network.attr('node', shape='circle', fixedsize='true', width='0.3', \
             label=None, fontsize='9', style='filled', fontcolor='white')

for node in graph['links']:
    for nbr in graph['links'][node]:
        # We'll remove the 'cube' and 'cluster' strings from the label to avoid clutter
        node=str.replace(str.replace(node,'cube',''),'cluster','')
        nbr=str.replace(str.replace(nbr,'cube',''),'cluster','')
        network.edge(node, nbr)
        
# Colour the nodes based on the average x value of points in the cluster
norm = colors.Normalize(vmin = df2['Points'].min(), vmax = df2['Points'].max())
clrmap=cm.ScalarMappable(norm = norm, cmap = cm.viridis)

for node in graph['nodes']:
    avg_pts=np.average([df2.iloc[i]['Points'] for i in graph['nodes'][node]])
    color=clrmap.to_rgba(avg_pts)
    color=colorsys.rgb_to_hsv(color[0], color[1], color[2])
    node=str.replace(str.replace(node,'cube',''),'cluster','')
    network.node(node, fillcolor="%f, %f, %f" % (color[0], color[1], color[2]))
    
# Display the graph
network

In [ ]:
%%capture
mapper.visualize(graph, path_html="premier_league_players.html",
                 title="Player Data")